In [1]:
import scanpy as sc
import spapros as sp
import numpy as np
import pandas as pd
from matplotlib.pyplot import figure
import pickle
import os
import scipy

figure(figsize=(15, 10), dpi=300)

<Figure size 4500x3000 with 0 Axes>

<Figure size 4500x3000 with 0 Axes>

In [2]:
os.getpid()

35818

In [3]:
#sc.settings.verbosity = 0
sc.logging.print_header()
print(f"spapros=={sp.__version__}")

scanpy==1.9.1 anndata==0.8.0 umap==0.5.3 numpy==1.23.3 scipy==1.9.1 pandas==1.5.0 scikit-learn==1.1.2 statsmodels==0.13.2 python-igraph==0.9.11 pynndescent==0.5.7
spapros==0.1.1


In [4]:
adata = sc.read_h5ad("/scratch/nmq407/ds_obj_sce_neuron_h5ad_from_notebook")
adata

AnnData object with n_obs × n_vars = 107464 × 18576
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Index.10x', 'nCount_HTO', 'nFeature_HTO', 'Index.HTO', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'HTO_mcl_maxID', 'HTO_mcl_secondID', 'HTO_mcl_margin', 'HTO_mcl_classification', 'HTO_mcl_classification.global', 'hash.mcl.ID', 'doublet', 'predicted_dub_std', 'predicted_dub_cut', 'predicted_dub_all', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'strain', 'treatment', 'time', 'collection_date', 'isolation_date', 'batch', 'predicted.id', 'prediction.score.max', 'labels', 'polar_label', 'ident'
    uns: 'X_name'
    obsm: 'PCA', 'UMAP'
    layers: 'logcounts'

In [5]:
adata_00 = adata[adata.obs_names, adata.var_names]
adata_00.obs['celltype'] = adata.obs['polar_label']
adata_00.obsm['X_umap'] = adata.obsm['UMAP'].to_numpy()
adata_00.X = scipy.sparse.csr_matrix(adata_00.X)
sc.pp.filter_genes(adata_00, min_cells=1)
sc.pp.highly_variable_genes(adata_00, flavor="seurat_v3",n_top_genes=5000)
adata_00

/tmp/ipykernel_35818/2089171798.py:2: ImplicitModificationWarning: Trying to modify attribute `.obs` of view, initializing view as actual.
  adata_00.obs['celltype'] = adata.obs['polar_label']
/tools/anaconda/envs/nmq407/fgf1_ygg/lib/python3.8/contextlib.py:120: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  next(self.gen)


AnnData object with n_obs × n_vars = 107464 × 15766
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'Index.10x', 'nCount_HTO', 'nFeature_HTO', 'Index.HTO', 'nCount_spliced', 'nFeature_spliced', 'nCount_unspliced', 'nFeature_unspliced', 'HTO_mcl_maxID', 'HTO_mcl_secondID', 'HTO_mcl_margin', 'HTO_mcl_classification', 'HTO_mcl_classification.global', 'hash.mcl.ID', 'doublet', 'predicted_dub_std', 'predicted_dub_cut', 'predicted_dub_all', 'nCount_SCT', 'nFeature_SCT', 'SCT_snn_res.0.8', 'seurat_clusters', 'strain', 'treatment', 'time', 'collection_date', 'isolation_date', 'batch', 'predicted.id', 'prediction.score.max', 'labels', 'polar_label', 'ident', 'celltype'
    var: 'n_cells', 'highly_variable', 'highly_variable_rank', 'means', 'variances', 'variances_norm'
    uns: 'X_name', 'hvg'
    obsm: 'PCA', 'UMAP', 'X_umap'
    layers: 'logcounts'

In [6]:
adata_00.X

<107464x15766 sparse matrix of type '<class 'numpy.float32'>'
	with 494575233 stored elements in Compressed Sparse Row format>

In [7]:
X = adata_00.X
X = np.log1p(X)
X = np.expm1(X)
mean = X.mean(axis=0).A1
mean[mean == 0] = 1e-12
mean = np.log1p(mean)
np.any(mean == np.inf)

False

In [ ]:
X.mean(axis=0).A1

In [ ]:
adata_00.obsm['X_umap']

In [ ]:
sc.pl.umap(adata_00)

In [ ]:
selector = sp.se.ProbesetSelector(adata_00,
                                  n=100,
                                  celltype_key="celltype",
                                  verbosity=1,
                                  save_dir='blah/',
                                  n_jobs=40)

In [ ]:
selector.select_probeset()

In [ ]:
selector.probeset

In [ ]:
selected_set = selector.probeset.index[selector.probeset.selection].tolist()
selected_set

In [ ]:
masked_dotplot = sp.pl.masked_dotplot(adata_00, selector)
masked_dotplot.set_size_inches(18.5, 10.5)
masked_dotplot

In [ ]:
selector.plot_gene_overlap()

In [ ]:
evaluator = sp.ev.ProbesetEvaluator(adata_00, verbosity=2, results_dir='evaluate_anon')

In [ ]:
evaluator.evaluate_probeset(selected_set, set_id="Spapros")

In [ ]:
# Reference probesets
reference_sets = sp.se.select_reference_probesets(adata_00, n=100)

In [ ]:
for set_id, df in reference_sets.items():
    gene_set = df[df["selection"]].index.to_list()
    evaluator.evaluate_probeset(gene_set, set_id=set_id)


In [ ]:
evaluator.summary_results#.index.tolist()

In [ ]:
import matplotlib as mpl
mpl.rcParams['figure.dpi'] = 300

In [ ]:
# evaluator.summary_statistics("Spapros")
evaluator.plot_summary()

In [ ]:
figure(figsize=(30, 20), dpi=300)
evaluator.plot_confusion_matrix()